<a href="https://colab.research.google.com/github/No1Talent/DLSelfDrivingCars/blob/master/Self_Driving_Car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets # handling deep learning models, image transformations, and datasets

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print('Using gpu: %s ' % torch.cuda.is_available())

Using gpu: True 


# Data Prep

Initial Import

```
# download and unzip
!wget https://d17h27t6h515a5.cloudfront.net/topher/2016/December/584f6edd_data/data.zip
!unzip data.zip
%ls

# remove unnecessary
!rm data.zip
!rm -r __MACOSX

# save data to MyDrive
from google.colab import drive
drive.mount('/content/drive')
!mv data "/content/drive/MyDrive/"
```


In [9]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir('/content/gdrive/MyDrive/')

%pwd

Mounted at /content/gdrive


'/content/gdrive/MyDrive'

In [14]:
import csv
from torch.utils.data import random_split

# Read the CSV file and skip the header
with open('data/driving_log.csv') as csvfile:
    samples = list(csv.reader(csvfile))[1:] # skip the header

# Split the data into training and validation sets
train_len = int(0.8 * len(samples))
train_samples, validation_samples = random_split(samples, [train_len, len(samples) - train_len])

In [69]:
len(samples), samples[:1]

(8036,
 [['IMG/center_2016_12_01_13_30_48_287.jpg',
   ' IMG/left_2016_12_01_13_30_48_287.jpg',
   ' IMG/right_2016_12_01_13_30_48_287.jpg',
   ' 0',
   ' 0',
   ' 0',
   ' 22.14829']])

# Image Augmentation

In [19]:
import cv2
import numpy as np

# perform image augmentation by flipping images horizontally with a 50% probability.
def augment(img_name, angle):
    name = f'data/IMG/{img_name.split("/")[-1]}'
    current_image = cv2.imread(name)[65:-25, :, :]

    if np.random.rand() < 0.5:
        current_image = cv2.flip(current_image, 1)
        angle = -angle

    return current_image, angle

In [28]:
# TRANSFORMING AND AUGMENTING IMAGES
from torchvision import models,transforms,datasets
from torchvision.transforms import v2

transformations = v2.Compose([
    # v2.ToImage(),  # Convert to tensor, only needed if you had a PIL image
    # v2.ToDtype(torch.uint8, scale=True),  # optional, most input are already uint8 at this point
    # v2.Resize(size=(224, 224), antialias=True), # v2.RandomResizedCrop(size=(224, 224), antialias=True)
    v2.RandomHorizontalFlip(p=0.5),
    v2.ToDtype(torch.float32, scale=True),  # Normalize expects float input
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])

# antialias=True to smooth the edges of images or graphics to reduce aliasing artifacts, such as jagged edges or flickering

'''
from torch.utils.data import DataLoader
import torchvision.transforms as transforms

transformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])
'''

'\nfrom torch.utils.data import DataLoader\nimport torchvision.transforms as transforms\n\ntransformations = transforms.Compose([transforms.Lambda(lambda x: (x / 255.0) - 0.5)])\n'

# Data Loading and Batching
created a custom dataset by subclassing data.Dataset??

In [29]:
class Dataset(data.Dataset):
    def __init__(self, samples, transform=None):
        self.samples = samples
        self.transform = transform
    # retrieves items based on the given index, applying augmentation as well
    def __getitem__(self, index):
        batch_samples = self.samples[index]
        steering_angle = float(batch_samples[3])

        center_img, angle_center = augment(batch_samples[0], steering_angle)
        left_img, angle_left = augment(batch_samples[1], steering_angle + 0.4)
        right_img, angle_right = augment(batch_samples[2], steering_angle - 0.4)

        center_img = self.transform(center_img)
        left_img = self.transform(left_img)
        right_img = self.transform(right_img)

        return (center_img, angle_center), (left_img, angle_left), (right_img, angle_right)

    def __len__(self):
        return len(self.samples)

In [ ]:
class CarDataset3Img(data.Dataset):
    """
    Dataset wrapping images and target steering_angle.
    """

    def __init__(self, X, y, data_dir, transform=None):

        self.X = X
        self.y = y
        self.data_dir = data_dir
        self.transform = transform

    def __getitem__(self, index):
        center, left, right = self.X[index]
        steering_angle = self.y[index]
        image_left, steering_angle_left = augument_img(load_image(self.data_dir, left), steering_angle + 0.1)

        image_center, steering_angle_center = augument_img(load_image(self.data_dir, center), steering_angle)

        image_right, steering_angle_right = augument_img(load_image(self.data_dir, right), steering_angle - 0.1)


        if self.transform is not None:
            image_left = self.transform(image_left)
            image_center = self.transform(image_center)
            image_right = self.transform(image_right)

        #label = torch.from_numpy(self.y_train[index])
        return (image_center, steering_angle_center), (image_left, steering_angle_left), (image_right, steering_angle_right)

    def __len__(self):
        return self.X.shape[0]


In [30]:
dset_train = Dataset(train_samples, transformations)
imagenet_data = torchvision.datasets.ImageNet('path/to/imagenet_root/')
loader_train = data.DataLoader(dset_train, batch_size=64, shuffle=True, num_workers=6)
# params = {'batch_size': 32, 'shuffle': True, 'num_workers': 4}
# loader_train = data.DataLoader(dset_train, **params)

dset_valid = Dataset(validation_samples, transformations)
loader_valid = data.DataLoader(dset_valid, batch_size=64, shuffle=False, num_workers=6)

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [67]:
len(dset_valid)

1608

In [65]:
# Get the next batch from the data loader
# data_try = next(iter(loader_valid))
len(data_try)
# [[],[],[]]

IndexError: ignored

In [60]:
# Check the structure of the returned data
data_try[0][0].shape, data_try[0][1].shape, data_try[1][0].shape, data_try[1][1].shape,data_try[2][0].shape, data_try[2][1]

(torch.Size([64, 3, 224, 224]),
 torch.Size([64]),
 torch.Size([64, 3, 224, 224]),
 torch.Size([64]),
 torch.Size([64, 3, 224, 224]),
 tensor([-0.4000,  0.4000,  0.4598,  0.4693,  0.4000,  0.4693,  0.3382,  0.4000,
         -0.4000,  0.4882,  0.4000,  0.4882,  0.4000,  0.4218,  0.4000,  0.3957,
          0.6686, -0.4000,  0.4000,  0.2234,  0.3095, -0.4000, -0.1947, -0.4000,
         -0.4000,  0.4000, -0.4000, -0.6781, -0.2521,  0.4000, -0.4000,  0.0416,
         -0.4000, -0.8395, -0.3095, -0.4598,  0.4693,  0.4000,  0.3861,  0.4787,
         -0.4000,  0.4000, -0.4000,  0.0512, -0.5452,  0.4000,  0.1851, -0.5357,
         -0.6117, -0.4000, -0.4000,  0.4000, -0.4000, -0.3000,  0.4000,  0.4000,
         -0.4000,  0.6971, -0.4000, -0.4000,  0.4000, -0.2330,  0.4000,  0.4000],
        dtype=torch.float64))

In [42]:
# take a batch to check
inputs_try, ,labels_try = next(iter(loader_valid))

# check input shape

ValueError: ignored

# Model Architecture

 ### NVIDIA Model
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)
    
    the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem.
    
### Simple Model

```
  (module): CarSimpleModel (
    (conv_layers): Sequential (
      (0): Conv2d(3, 24, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (1): ELU (alpha=1.0)
      (2): Conv2d(24, 48, kernel_size=(3, 3), stride=(2, 2), bias=False)
      (3): MaxPool2d (size=(4, 4), stride=(4, 4), dilation=(1, 1))
      (4): Dropout (p = 0.25)
    )
    (linear_layers): Sequential (
      (0): Linear (3648 -> 50)
      (1): ELU (alpha=1.0)
      (2): Linear (50 -> 10)
      (3): Linear (10 -> 1)
    )
  )
```

In [31]:
import torch.nn as nn
import torch.nn.init as init
import torch.optim as optim
import torch.nn.functional as F

class NetworkDense(nn.Module):

    def __init__(self):
        super(NetworkDense, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(36, 48, 5, stride=2),
            nn.ELU(),
            nn.Conv2d(48, 64, 3),
            nn.ELU(),
            nn.Conv2d(64, 64, 3),
            nn.Dropout(0.25)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=64 * 2 * 33, out_features=100),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )

    def forward(self, input):
        input = input.view(input.size(0), 3, 70, 320)
        output = self.conv_layers(input)
        print(output.shape)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output


class NetworkLight(nn.Module):

    def __init__(self):
        super(NetworkLight, self).__init__()
        self.conv_layers = nn.Sequential(
            nn.Conv2d(3, 24, 3, stride=2),
            nn.ELU(),
            nn.Conv2d(24, 48, 3, stride=2),
            nn.MaxPool2d(4, stride=4),
            nn.Dropout(p=0.25)
        )
        self.linear_layers = nn.Sequential(
            nn.Linear(in_features=48*4*19, out_features=50),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10),
            nn.Linear(in_features=10, out_features=1)
        )


    def forward(self, input):
        input = input.view(input.size(0), 3, 70, 320)
        output = self.conv_layers(input)
        print(output.shape)
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output



```
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
#from torchnet.meter import AverageValueMeter
import torch.backends.cudnn as cudnn


class CarModel(nn.Module):
    def __init__(self):
        super(CarModel, self).__init__()
        self.conv_layers = nn.Sequential(
            # input is batch_size x 3 x 66 x 200
            nn.Conv2d(3, 24, 5, stride=2, bias=False),
            #nn.ELU(0.2, inplace=True),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2, bias=False),
            nn.ELU(),
            nn.BatchNorm2d(36),
            
            nn.Conv2d(36, 48, 5, stride=2, bias=False),
            nn.ELU(),
            nn.BatchNorm2d(48),
            
            nn.Conv2d(48, 64, 3, stride=1, bias=False),
            nn.ELU(),
            nn.BatchNorm2d(64),
            
            nn.Conv2d(64, 64, 3, stride=1, bias=False),
            nn.ELU(),
            nn.Dropout(p=0.4)
        )
        self.linear_layers = nn.Sequential(
            #input from sequential conv layers
            nn.Linear(in_features=64*1*18, out_features=100, bias=False),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50, bias=False),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10, bias=False),
            nn.ELU(),
            nn.Linear(in_features=10, out_features=1, bias=False))
        self._initialize_weights()
        
    # custom weight initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                init.normal(m.weight, mean=0, std=0.02)
            elif isinstance(m, nn.BatchNorm2d):
                init.normal(m.weight, mean=1, std=0.02)
                init.constant(m.bias, 0)

    def forward(self, input):
        output = self.conv_layers(input)
        output = output.view(output.size(0), 64*1*18)
        output = self.linear_layers(output)
        return output
    
class CarDenseModel(nn.Module):
    def __init__(self):
        super(CarDenseModel, self).__init__()
        self.conv_layers = nn.Sequential(
            # input is batch_size x 3 x 66 x 200
            nn.Conv2d(3, 24, 5, stride=2, bias=False),
            #nn.ELU(0.2, inplace=True),
            nn.ELU(),
            nn.Conv2d(24, 36, 5, stride=2, bias=False),
            nn.ELU(),
            nn.BatchNorm2d(36),
            
            nn.Conv2d(36, 48, 5, stride=2, bias=False),
            nn.ELU(),
            nn.BatchNorm2d(48),
            
            nn.Conv2d(48, 64, 3, stride=1, bias=False),
            nn.ELU(),
            nn.BatchNorm2d(64),
            
            nn.Conv2d(64, 64, 3, stride=1, bias=False),
            nn.ELU(),
            nn.Dropout(p=0.4)
        )
        self.linear_layers = nn.Sequential(
            #input from sequential conv layers
            nn.Linear(in_features=64*1*18+9*32*99, out_features=100, bias=False),
            nn.ELU(),
            nn.Linear(in_features=100, out_features=50, bias=False),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10, bias=False),
            nn.ELU(),
            nn.Linear(in_features=10, out_features=1, bias=False))
        
        self.shortcut = nn.Sequential(
                nn.Conv2d(3, 9, 3, stride=2, bias=False),
                nn.BatchNorm2d(9)
            )
        
        self._initialize_weights()
        
    # custom weight initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                init.normal(m.weight, mean=0, std=0.02)
            elif isinstance(m, nn.BatchNorm2d):
                init.normal(m.weight, mean=1, std=0.02)
                init.constant(m.bias, 0)

    def forward(self, input):
        output = self.conv_layers(input)
        
        output = output.view(output.size(0), 64*1*18)
        input_shortcut = self.shortcut(input)
        
        input_shortcut = input_shortcut.view(input_shortcut.size(0), 9*32*99)
        output_sum = torch.cat((output, input_shortcut), 1).view(output.size(0), 64*1*18+9*32*99)
        output_sum = self.linear_layers(output_sum)
        return output_sum
    
    
class CarSimpleModel(nn.Module):
    def __init__(self):
        super(CarSimpleModel, self).__init__()
        self.conv_layers = nn.Sequential(
            # input is batch_size x 3 x 16 x 32
            nn.Conv2d(3, 24, 3, stride=2, bias=False),
            nn.ELU(),
            nn.Conv2d(24, 48, 3, stride=2, bias=False),
            nn.MaxPool2d(4, stride=4),
            nn.Dropout(p=0.25)
        )
        self.linear_layers = nn.Sequential(
            #input from sequential conv layers
            nn.Linear(in_features=48*4*19, out_features=50, bias=False),
            nn.ELU(),
            nn.Linear(in_features=50, out_features=10, bias=False),
            nn.Linear(in_features=10, out_features=1, bias=False),
        )
        self._initialize_weights()
        
    # custom weight initialization
    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                init.normal(m.weight, mean=0, std=0.02)
            elif isinstance(m, nn.BatchNorm2d):
                init.normal(m.weight, mean=1, std=0.02)
                init.constant(m.bias, 0)

    def forward(self, input):
        
        input = input.view(input.size(0), 3, 75, 320)
        output = self.conv_layers(input)
        #print('size ' + str(output.size()))
        output = output.view(output.size(0), -1)
        output = self.linear_layers(output)
        return output    
```



In [32]:
model = NetworkLight()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

criterion = nn.MSELoss()

# Training and Validation Loop

In [38]:
max_epochs = 10
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

def to_device(datas, device):
    imgs, angles = datas
    return imgs.float().to(device), angles.float().to(device)

for epoch in range(max_epochs):
    model.to(device)

    # Training
    train_loss = 0
    model.train()
    for local_batch, (centers, lefts, rights) in enumerate(training_generator):
        # Transfer to GPU
        centers, lefts, rights = to_device(centers, device), to_device(lefts, device), to_device(rights, device)

        # Model computations
        optimizer.zero_grad()
        for data in [centers, lefts, rights]:
            imgs, angles = data
            outputs = model(imgs)
            loss = criterion(outputs, angles.unsqueeze(1))
            loss.backward()
            optimizer.step()
            train_loss += loss.item()

        if local_batch % 100 == 0:
            print(f'Epoch [{epoch+1}/{max_epochs}], Batch [{local_batch+1}], Train Loss: {train_loss/((local_batch+1)*3):.3f}')

    # Validation
    model.eval()
    valid_loss = 0
    with torch.no_grad():
        for local_batch, (centers, lefts, rights) in enumerate(validation_generator):
            # Transfer to GPU
            centers, lefts, rights = to_device(centers, device), to_device(lefts, device), to_device(rights, device)

            # Model computations
            optimizer.zero_grad()
            for data in [centers, lefts, rights]:
                imgs, angles = data
                outputs = model(imgs)
                loss = criterion(outputs, angles.unsqueeze(1))
                valid_loss += loss.item()

            avg_valid_loss = valid_loss/(local_batch+1)
            if local_batch % 100 == 0:
                print(f'Epoch [{epoch+1}/{max_epochs}], Validation Batch [{local_batch+1}], Valid Loss: {avg_valid_loss:.3f}')


流式输出内容被截断，只能显示最后 5000 行内容。
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4, 19])
torch.Size([32, 48, 4

# Plot Errors

In [41]:
avg_valid_loss

0.13797200688471398

# Save Model to disk

In [39]:
state = {
        'model': model.module if device == 'cuda' else model,
        }

torch.save(state, 'model.h5')